In [108]:
file_path = 'saint-exupery-maly-ksiaze.txt'
MAX_FEATURES = 3700
EMBEDDING_DIM = 128
WINDOW_SIZE = 10
BATCH_SIZE = 32
EPOCHS = 10

In [109]:
with open(file_path) as file:
    lines = file.readlines()
lines = list(filter(lambda x: len(x.split(' ')) > 2, lines))
lines[:5]

['Antoine de Saint-Exupéry\n',
 'tłum. Agata Kozak\n',
 'Proszę dzieci, aby mi wybaczyły, że dedykuję tę książkę dorosłemu. Mam niebagatelne usprawiedliwienie: ten dorosły to mój najlepszy przyjaciel na świecie. Mam też inne usprawiedliwienie: ten dorosły potrafi wszystko zrozumieć, nawet książki dla dzieci. Mam jeszcze trzecie usprawiedliwienie: ten dorosły mieszka we Francji, gdzie cierpi głód i chłód. Bardzo potrzebuje pociechy. Jeżeli wszystkie te powody okażą się niewystarczające, chętnie zadedykuję tę książkę dziecku, którym był kiedyś ten dorosły. Wszyscy dorośli byli najpierw dziećmi. (Ale niewielu z nich o tym pamięta.) Poprawiam więc dedykację:\n',
 'z czasów, kiedy był małym chłopcem\n',
 'Kiedy miałem sześć lat, zobaczyłem pewnego razu wspaniały obrazek w książce o dżungli zatytułowanej Historie prawdziwe. Przedstawiał węża boa połykającego lwa. Oto kopia tego rysunku:\n']

In [110]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(
    num_words=MAX_FEATURES,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n—',
)
tokenizer.fit_on_texts(lines)
word_index = tokenizer.word_index
index_word = tokenizer.index_word
len(word_index)

3704

In [111]:
from keras.layers import Input, Embedding, Dot, Reshape, Activation
from keras import Model

word_input = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(
    MAX_FEATURES + 1,
    EMBEDDING_DIM
)(word_input)

context_input = Input(shape=(1, ), dtype='int32')
context_embedding  = Embedding(
    MAX_FEATURES + 1,
    EMBEDDING_DIM,
)(context_input)

dot = Dot(axes=2)([word_embedding, context_embedding])
dot = Reshape((1,), input_shape=(1, 1))(dot)
activation = Activation('sigmoid')(dot)

model = Model(inputs=[word_input, context_input], outputs=activation)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 1, 128)       473728      input_13[0][0]                   
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 1, 128)       473728      input_14[0][0]                   
____________________________________________________________________________________________

In [112]:
from keras.preprocessing.sequence import skipgrams

sequences = tokenizer.texts_to_sequences(lines)
X, y = [], []

for sequence in sequences:
    couples, labels = skipgrams(
        sequence, 
        MAX_FEATURES+1, 
        window_size=WINDOW_SIZE, 
        negative_samples=1.0, 
        shuffle=True,
        seed=42
    )
    X.extend(couples), y.extend(labels)
X[:5], y[:5]

([[703, 2604], [704, 705], [704, 1884], [704, 3006], [705, 1776]],
 [0, 1, 0, 0, 0])

In [113]:
import numpy as np

def data_generator(batch_size):
    words, contexts, labels = [], [], []
    while True:
        for i , (couple, label) in enumerate(zip(X, y)):
            word = np.array([couple[0]])
            context = np.array([couple[1]])
            words.append(word)
            contexts.append(context)
            labels.append([label])
            if (i + 1) % batch_size == 0:
                yield [np.asarray(words), np.asarray(contexts)], np.asarray(labels)
                words = []
                contexts = []
                labels = []

next(data_generator(2))

([array([[703],
         [704]]),
  array([[2604],
         [ 705]])],
 array([[0],
        [1]]))

In [114]:
steps = len(X)/BATCH_SIZE

model.fit(
    data_generator(BATCH_SIZE),
    epochs = EPOCHS,
    steps_per_epoch = steps,
)

Epoch 1/10
10297/10296 [==============================] - 96s 9ms/step - loss: 0.5275 - accuracy: 0.7554
Epoch 2/10
10297/10296 [==============================] - 95s 9ms/step - loss: 0.4617 - accuracy: 0.8005
Epoch 3/10
10297/10296 [==============================] - 96s 9ms/step - loss: 0.3953 - accuracy: 0.8343
Epoch 4/10
10297/10296 [==============================] - 93s 9ms/step - loss: 0.3184 - accuracy: 0.8747
Epoch 5/10
10297/10296 [==============================] - 93s 9ms/step - loss: 0.2491 - accuracy: 0.9077
Epoch 6/10
10297/10296 [==============================] - 93s 9ms/step - loss: 0.1960 - accuracy: 0.9294
Epoch 7/10
10297/10296 [==============================] - 95s 9ms/step - loss: 0.1589 - accuracy: 0.9430
Epoch 8/10
10297/10296 [==============================] - 94s 9ms/step - loss: 0.1342 - accuracy: 0.9503
Epoch 9/10
10297/10296 [==============================] - 94s 9ms/step - loss: 0.1180 - accuracy: 0.9549
Epoch 10/10
10297/10296 [==============================

In [115]:
word2vec={}
vectors = model.get_weights()[0] # weights without bias
for word, i in word_index.items():
    if i < MAX_FEATURES:
        word2vec[word]=list(vectors[i, :])

word2vec['dzieci']

[-0.057225496,
 -0.20672044,
 -0.5354497,
 -0.23587765,
 0.047212545,
 -0.13578546,
 0.105006434,
 0.2249624,
 0.60114485,
 0.011181073,
 0.028876191,
 -0.2150197,
 -0.19229287,
 -0.42304903,
 -0.3870341,
 -0.26633513,
 0.20992306,
 -0.26789796,
 0.63235205,
 0.18417272,
 -0.23758674,
 -0.086795725,
 0.40518805,
 -0.26184282,
 0.19725634,
 -0.08173382,
 0.07478525,
 -0.32972407,
 -0.07596091,
 -0.1818065,
 -0.39132196,
 0.12636523,
 -0.37860554,
 -0.09653058,
 0.09106132,
 -0.28574094,
 0.16958746,
 0.3122325,
 0.037383165,
 -0.17949788,
 -0.64787537,
 0.3050659,
 0.9507067,
 -0.07905616,
 0.22398783,
 0.59930265,
 -0.052212063,
 0.10895566,
 0.27813455,
 0.17503741,
 0.12873241,
 -0.24379854,
 -0.3741396,
 -0.9290059,
 -0.28661683,
 -0.18567401,
 -0.2175693,
 -0.13793968,
 -0.06619417,
 -0.21876732,
 0.085079394,
 0.4730686,
 -0.52085024,
 0.13513213,
 -0.09339047,
 -0.141848,
 -0.12019018,
 -0.61028796,
 -0.103933506,
 -0.2973206,
 0.46471804,
 -0.12984988,
 -0.044988733,
 -0.2344372

In [116]:
def avg_sentence_vector(words, word2vec):
    vec = np.zeros((EMBEDDING_DIM,), dtype="float32")
    for word in words:
        if word in word2vec.keys():
            vec = np.add(vec, word2vec[word])
    return vec

In [117]:
from scipy.spatial.distance import cosine, euclidean
sentence1 = ['proszę', 'dzieci', 'aby', 'mi', 'wybaczyły']
sentence2 = ['proszę', 'dorośli', 'aby', 'mi', 'wybaczyły']

cosine(
    avg_sentence_vector(
        sentence1,
        word2vec
    ),
      avg_sentence_vector(
      sentence2,
      word2vec
))

0.06781631708145142

In [118]:
euclidean(
    avg_sentence_vector(
        sentence1,
        word2vec
    ),
      avg_sentence_vector(
      sentence2,
      word2vec
))

4.935206413269043

In [119]:
test_words = ['usprawiedliwienie', 'dorosły', 'potrafi', 'wszystko', 'zrozumieć', 'książki', 'dzieci']
test_words_idx = [word_index[word] for word in test_words]
test_words_idx

[484, 254, 709, 78, 257, 710, 209]

In [120]:
from sklearn.metrics import pairwise_distances
from collections import defaultdict

vecs = np.zeros((MAX_FEATURES, EMBEDDING_DIM))
for i in range(1, MAX_FEATURES):
    vecs[i] = word2vec[index_word[i]]
    
closest = defaultdict(list)
for idx in test_words_idx:
    dists = pairwise_distances(vecs, metric='cosine')
    closest_id = np.argsort(dists[idx, :])[1:5]
    for i in closest_id:
        closest[index_word[idx]].append((index_word[i], dists[idx, i]))
closest

defaultdict(list,
            {'usprawiedliwienie': [('trzecie', 0.2641702798591403),
              ('niebagatelne', 0.2794349904318243),
              ('wybaczyły', 0.3741950350028179),
              ('chłód', 0.38417133248781477)],
             'dorosły': [('poznał', 0.4007957122756466),
              ('człowieka', 0.4325554983507145),
              ('krawatach', 0.4454413330731455),
              ('powody', 0.4469661759184015)],
             'potrafi': [('usprawiedliwienie', 0.5316574419491084),
              ('usłyszeć', 0.5465419664365483),
              ('pochwały', 0.5729732027822341),
              ('dorosłemu', 0.600687459087724)],
             'wszystko': [('miesza', 0.5747617535688968),
              ('przyjemność', 0.5953941120551371),
              ('mimo', 0.6096148219102052),
              ('przeliczyli', 0.6408985918467736)],
             'zrozumieć': [('błahostkę', 0.5295959366860877),
              ('zrozumie', 0.5345679409638675),
              ('książkę', 0.53573758

In [122]:
model.save_weights('word2vec.h5', overwrite=True)